In [ ]:
list ( set([1,2,3]) - set([2]) )

In [ ]:
# data_dir = '/home/demitau/data_duss'
# !DATA_DUSS=$data_dir

In [ ]:
from os.path import join as pjoin

import mne
import os,sys

import os,sys
sys.path.append('/home/demitau/ownCloud/Current/NeuroSci/MSCA_sci/code/data_proc')
subjects_dir = os.path.expandvars('$SUBJECTS_DIR')
from globvars import data_dir

# if os.environ.get('DATA_DUSS') is not None:
#     data_dir = os.path.expandvars('$DATA_DUSS')
# else:
#     data_dir = '/home/demitau/data'
    
raws = {}

rawnames = ['S01_off_hold', 'S01_on_hold']; data_type = 'hold'
#fnames_noext = ['S01_off_move', 'S01_on_move']; data_type = 'move'
for rawname in rawnames:
    #fname = rawname + '_hires-raw.fif'
    fname = rawname + '_resample_notch_raw.fif'
    print(fname)
    fname_full = os.path.join(data_dir,fname)
    if not os.path.exists(fname_full):
        print('Warning: path does not exist! {}'.format(fname_full))
        continue

#fname = 'S01_off_hold.mat'
    f = mne.io.read_raw_fif(fname_full, None)
    
    raws[rawname] = f 

In [ ]:
from autoreject import AutoReject,get_rejection_threshold,Ransac
ar = AutoReject()
# epochs_clean = ar.fit_transform(epochs)  
# reject = get_rejection_threshold(epochs)  
# rsc = Ransac()
# epochs_clean = rsc.fit_transform(epochs)  

# ar.fit_transform?

# AutoReject?

In [ ]:
totnbins = 300 * 256 
3 / totnbins

In [ ]:
1e-3 * totnbins

In [ ]:
fine_cal_file = os.path.join(data_dir, 'sss_cal.dat')
crosstalk_file = os.path.join(data_dir,  'ct_sparse.fif')

raw = raws['S01_on_hold']
raw_sss = mne.preprocessing.maxwell_filter(raw , cross_talk=crosstalk_file,
                                           calibration=fine_cal_file, coord_frame='meg')

In [ ]:
data_type = ''
move_found = 0
hold_found = 0
for k in raws.keys():
    if k.find('move') >= 0:
        move_found = 1
    elif k.find('hold') >= 0:
        hold_found = 1
        
if move_found:
    if hold_found:
        data_type = 'mix'
    else:
        data_type = 'move'
else:
    data_type = 'hold'

print(data_type)

In [ ]:
info = f.info
chnames = info['ch_names']
#print(chnames)

emg_inds = [i for i,s in enumerate(f.info['ch_names']) if 0 <= s.find('EMG') ]
emgkil_inds = [i for i,s in enumerate(f.info['ch_names']) if 0 <= s.find('_kil') ]
lfp_inds = [i for i,s in enumerate(f.info['ch_names']) if 0 <= s.find('LFP') ]
lfpl_inds = [i for i,s in enumerate(f.info['ch_names']) if 0 <= s.find('LFPL') ]  # left STN LFP
lfpr_inds = [i for i,s in enumerate(f.info['ch_names']) if 0 <= s.find('LFPR') ]  # right STN LFP
eye_vert_name = 'EOG127'
eye_hor_name = 'EOG128'
eog_inds = [i for i,s in enumerate(chnames) if 0 <= s.find('EOG') ]
right_edc = 'EMG061'
right_fds = 'EMG062'
left_edc = 'EMG063'
left_fds = 'EMG064'

## playing with grids

In [ ]:
import pymatreader as pym
import pandas as pd
#head_fname = os.path.join(data_dir, 'headmodel_grid_S01.mat')
head_fname = os.path.join(data_dir, 'Hilbert_headmodels', 's011.mat')

#head = pd.read_hdf(head_fname)

##import h5py
#fhead = h5py.File(head_fname )
fhead =  pym.read_mat(head_fname)

import numpy as np

In [ ]:
def makeStr(smth):
    yy = list ( smth )
    a = map( lambda x:  chr(x[0]) , yy)    
    return ''.join(a)      

In [ ]:
#dict(fhead )

In [ ]:
refs = dict( fhead['#refs#'] )
print( refs )
makeStr( refs['b'] )

In [ ]:
fhead['hdm'].keys()

In [ ]:
print( makeStr(fhead['hdm']['type'] ) )
print( makeStr (fhead['hdm']['unit'] )  )

In [ ]:
for k,v in fhead.items():
    print(k)

In [ ]:
dict(fhead['hdm']['bnd'] )

In [ ]:
dict( fhead['hdm']['bnd']['cfg'] )

In [ ]:
pos = np.array( fhead['hdm']['bnd']['pos']  )
print(pos.shape, pos)

In [ ]:
tri = np.array( fhead['hdm']['bnd']['tri']  )
print(pos.shape, pos)

In [ ]:
dict( fhead['hdm']['bnd']['cfg']['callinfo'] )

In [ ]:
fhead['hdm']['bnd']['pos']

In [ ]:
tri_ = np.array( fhead['hdm']['bnd']['tri'] , dtype=np.int)
tri = tri_ - 1 #matlab 2 python
print(tri.shape, tri)

In [ ]:
fhead['mni_aligned_grid'].keys()

In [ ]:
dims = np.array( fhead['mni_aligned_grid']['dim'] , dtype=np.int)[:,0]
print(dims, np.prod(dims))

In [ ]:
fhead['mni_aligned_grid']['cfg'].keys()

In [ ]:
# probably tell whether a given grid point is inside the brain or outside
inside = np.array ( fhead['mni_aligned_grid']['inside']   )
print( np.sum( inside ) )
inside_ = inside.reshape( dims)
inside_[:,0,0]

## old

In [ ]:
gridpos = np.array( fhead['mni_aligned_grid']['pos'] )
gridpos.shape

In [ ]:
makeStr( fhead['mni_aligned_grid']['unit'] )

In [ ]:
# srcstr | instance of SourceSpaces :   If string, should be a source space filename. Can also be an instance of loaded or generated SourceSpaces.
# bemdict | str : Filename of the BEM (e.g., “sample-5120-5120-5120-bem-sol.fif”) to use, or a loaded sphere model (dict).
k = list(raws.keys() ) [0]
f = raws[k]


bem_model = mne.make_bem_model('S01', subjects_dir=subjects_dir)
bem_sol = mne.make_bem_solution(bem_model)

In [ ]:
fwd = mne.make_forward_solution(f.info, None, src, bem_sol) 

In [ ]:
os.environ["SUBJECTS_DIR"] = data_dir

In [ ]:
src = mne.setup_source_space('sample', spacing='oct6') 

In [ ]:
picks = mne.pick_types(f.info, meg='grad', exclude='bads')


In [ ]:
fwd = mne.read_forward_solution(fname_fwd)

In [ ]:
pos.shape

In [ ]:
gridpos.shape

In [ ]:
x=pos[0,:]; y =pos[1,:]; z=pos[2,:]
gridskip = 3
xg = gridpos[0,::3]; yg = gridpos[1,::3]; zg=gridpos[2,::3]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
tri.shape

In [ ]:
%matplotlib qt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(1, 2, 1, projection='3d')

ax.plot_trisurf(x,y,z,triangles=tri.T)
ax.scatter(xg,yg,zg,c='green',alpha=0.5)

In [ ]:
atlas_dir = os.path.join( data_dir, 'HMAT')
M1fname_left = os.path.join(atlas_dir,'HMAT_Left_M1.nii')

In [ ]:
 import nibabel as nib
img = nib.load(M1fname_left)


In [ ]:
img.shape

In [ ]:
hdr = img.header

In [ ]:
data = img.get_fdata()

In [ ]:
np.sum( data ) # perhabs only zeros and ones there?

In [ ]:
nz = np.where(data > 0)

1. I need convert to MNI from Talairach...
1. Somehow make indices correspond to coordinates
1. Make sure the points are actually located on the brain surface  and that brain model I have corresponds to something
1. Use coordinates for source reconstruction
1. How do I 
  
  1. mne.setup_source_space

In [ ]:
nz

In [ ]:
np.mean(data)

In [ ]:
print(hdr)

In [ ]:
print(hdr['sform_code'])
#3 means RAS+ in Talairach atlas space

In [ ]:
# The segmentation algorithms described in The Boundary Element Model (BEM) produce either FreeSurfer surfaces or triangulation data in text. Before proceeding to the creation of the boundary element model, standard files for FreeSurfer surfaces must be present:

#     inner_skull.surf contains the inner skull triangulation.
#     outer_skull.surf contains the outer skull triangulation.
#     outer_skin.surf contains the head surface triangulation.



mne.utils.set_config("SUBJECTS_DIR", '$HOME/data', set_env=True)
conductivity = (0.3,)  # for single layer
model = mne.make_bem_model('sample',conductivity=conductivity)  
bem_sol = make_bem_solution(model)
# The surface based source space src contains two parts, one for the left hemisphere (4098 locations) 
# and one for the right hemisphere (4098 locations). 
src = mne.setup_source_space(subject, spacing='oct6',
                             subjects_dir=subjects_dir, add_dist=False)
fwd = make_forward_solution(raw.info, fname_trans, src, bem_sol, meg=True, eeg=False, njobs=8, mindist=5.)  
inv = mne.minimum_norm.make_inverse_operator(raw.info, fwd, cov, loose=0.2)  
stc = mne.minimum_norm.apply_inverse(evoked, inv, lambda2=1. / 9.)  


# https://mne.tools/dev/auto_examples/inverse/plot_lcmv_beamformer.html?highlight=beamforming
#use_forward = mne.convert_forward_solution(forward, force_fixed=True)
use_forward = forward
filters = make_lcmv(evoked.info, use_forward, data_cov, reg=0.05,
                        noise_cov=noise_cov, pick_ori=pick_ori,
                        weight_norm='unit-noise-gain', rank=None)
# apply this spatial filter to source-reconstruct the evoked data
stc = apply_lcmv(evoked, filters, max_ori_out='signed')


# Computing LCMV solutions for time-frequency windows in a label in source
# space for faster computation, use label=None for full solution
stcs = tf_lcmv(epochs, forward, noise_covs, tmin, tmax, tstep, win_lengths,
               freq_bins=freq_bins, subtract_evoked=subtract_evoked,
               reg=data_reg, label=label, rank=None)

# Plotting source spectrogram for source with maximum activity.
# Note that tmin and tmax are set to display a time range that is smaller than
# the one for which beamforming estimates were calculated. This ensures that
# all time bins shown are a result of smoothing across an identical number of
# time windows.
plot_source_spectrogram(stcs, freq_bins, tmin=tmin_plot, tmax=tmax_plot,
                        source_index=None, colorbar=True)